### ***Logic***

##### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
##### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
     * Không quá 72h.
##### Backlog lấy
##### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
     * HCM -> HN: 12h.
     * Các tỉnh còn lại: 32h.
##### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
     * Giao hàng tại HCM HN: 36h.
     * Giao hàng tại các tỉnh còn lại: 56h.

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiKien',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[['TGDongKien', 'TGCapNhat', 'TGNhanKien', 'TGKetThuc']].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
# join with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien']], on='MaDH', how='left')

# classify types of backlog
# delivery backlog (backlog giao)
delivery = data[data['TrangThai'].isin([
    'Đang giao hàng', 'Giao không thành công',
    'Chờ xác nhận giao lại'
]) | ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] == data['KhoHienTai'])) |\
                          ((data['TrangThai'] == 'Tạo thành công') & ((data['ThoiGianKetThucLay'] != '<nil>') ^\
                          (data['ThoiGianKetThucGiao'] != '<nil>')))]
delivery['LoaiBacklog'] = 'Xử lý giao'
delivery['N0'] = delivery[delivery['KhoLay'] == delivery['KhoHienTai']]['ThoiGianKetThucLay']
delivery['N0'] = delivery[~(delivery['KhoLay'] == delivery['KhoHienTai'])]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(days=(120/24))
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(days=9999))


# return backlog (backlog trả)
returned = data[data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn không thành công'
    ]) | ((data['TrangThai'] == 'Trả hàng') & (((data['KhoTra'] != '<nil>') & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'] == '<nil>') & (data['KhoLay'] == data['KhoHienTai'])))) |\
          ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucGiao'] != '<nil>'))]
returned['LoaiBacklog'] = 'Xử lý trả'
returned['N0'] = returned[returned['KhoHienTai'] == returned['KhoGiao']]['ThoiGianKetThucGiao']
returned['N0'] = returned[~(returned['KhoHienTai'] == returned['KhoGiao'])]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(days=(72/24))
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(days=9999))


# pick_backlog (backlog lấy)
pickup = data[data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])]
pickup['LoaiBacklog'] = 'Xử lý lấy'
shopee = ['18692']
sendo = ['1539', '1160902', '1160904', '1160905']
tiki = ['1367']
lazada = ['1041351', '9794']
floors = shopee + sendo + tiki + lazada
pickup['N0'] = pickup[pickup['MaKH'].isin(floors)]['ThoiGianTao']
pickup['N0'] = pickup[~pickup['MaKH'].isin(floors)]['ThoiGianTaoChuyenDoi']
pickup['N+'] = pickup['N0'] + pd.Timedelta(days=3)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(days=9999))

# transit_backlog (backlog luân chuyển)
transit = data[(data['TrangThai'] == 'GHN đã lấy') |\
                         ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] != data['KhoHienTai']))]

transit['LoaiBacklog'] = 'Xử lý luân chuyển'
transit['N0'] = transit['ThoiGianKetThucLay']
transit['N+'] = transit[(transit['DenTinh'] == 'Hà Nội') | (transit['DenTinh'] == 'Hồ Chí Minh')]['N0'] + pd.Timedelta(days=12/24)
transit['N+'] = transit[~((transit['DenTinh'] == 'Hà Nội') | (transit['DenTinh']== 'Hồ Chí Minh'))]['N0'] + pd.Timedelta(days=32/24)
transit['Aging'] = (datetime.today() - transit['N+']).fillna(pd.Timedelta(days=9999))


# returned_transit_backlog (backlog luân chuyển trả hàng)
returned_transit = export[export['TrangThai'].isin([
    'Trả hàng', 'Chờ trả hàng'
]) & (((data['KhoTra'] == '<nil>') & (data['KhoLay'] != data['KhoHienTai'])) |\
      ((data['KhoTra'] != '<nil>') & (data['KhoTra'] != data['KhoHienTai'])))]
returned_transit['LoaiBacklog'] = 'Xử lý luân chuyển trả'
returned_transit['N0'] = returned_transit['ThoiGianKetThucGiao']
returned_transit['N+'] = returned_transit[
    (returned_transit['DenTinh'] == 'Hà Nội') | (returned_transit['DenTinh'] == 'Hồ Chí Minh')]['N0'] + pd.Timedelta(days=36/24)
returned_transit['N+'] = returned_transit[
    ~((returned_transit['DenTinh'] == 'Hà Nội') | (returned_transit['DenTinh'] == 'Hồ Chi Minh'))
]['N0'] + pd.Timedelta(days=56/24)
returned_transit['Aging'] = (datetime.today() - returned_transit['N+']).fillna(pd.Timedelta(days=9999))

# composite data
data = delivery.append([returned, pickup, transit, returned_transit])
# data['ToanTrinh'] = data[data['MaKH'].isin(floors)]['N0'] - datetime.today()
data


,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,LoaiBacklog,N0,N+,Aging
40,FHPAFQS4,KV1293243226366,2020-08-01 10:57:01,2020-08-01 10:57:00,2020-08-01 16:10:36,2020-08-04 09:17:38,NaT,2020-08-04 23:00:00,NaT,NaN,Quận Hoàng Mai,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 02,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,NaN,Đang giao hàng,500,1x1x1,518873,NaN,NaN,2 Ngày,1,1,0,0,1714228,9999,'ptlink',2267105,20NRU94012,2329.0,2275.0,2020-08-03 14:07:25,Xử lý giao,2020-08-03 14:07:25,2020-08-08 14:07:25,11 days 02:05:30.895216
44,8FH34YR14,5f26eb3bd47b3c00017f48d8_37,2020-08-02 23:39:41,2020-08-03 08:00:00,2020-08-14 13:52:23,2020-08-18 10:16:58,NaT,2020-08-20 08:18:00,NaT,NaN,Huyện Yên Mỹ,Hưng Yên,B,Quận 4,Hồ Chí Minh,E,Bưu Cục Yên Mỹ-Hưng Yên,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Yên Mỹ-Hưng Yên,Đang giao hàng,1000,10x10x10,2522198,NaN,'xem hàng ko nhận tt phí ship 30k- nga ',3 Ngày,1,1,0,NaN,2045705,NaN,'SHOP TÚI XÁCH HƯNG YÊN',1268482,20HBZ63012,2329.0,2275.0,2020-08-17 14:21:30,Xử lý giao,2020-08-17 14:21:30,2020-08-22 14:21:30,-3 days +01:51:25.895216
45,8FHHY6K9U,5f26652fd47b3c00017f4630_75,2020-08-02 14:18:18,2020-08-02 14:18:27,2020-08-15 07:58:51,2020-08-17 08:12:55,NaT,2020-08-21 06:24:00,NaT,NaN,Huyện Yên Mỹ,Hưng Yên,B,Quận 4,Hồ Chí Minh,E,Bưu Cục Yên Mỹ-Hưng Yên,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Yên Mỹ-Hưng Yên,Đang giao hàng,1000,10x10x10,2522198,'17/08/2020 10:24:47 - GHN-DFC1A0|THAY ĐỔI THÔ...,'xem hàng ko nhận tt phí ship 30k- nga ',3 Ngày,1,2,0,NaN,2045705,NaN,'SHOP TÚI XÁCH HƯNG YÊN',1268482,20VBP58349,2329.0,2275.0,2020-08-16 14:18:44,Xử lý giao,2020-08-16 14:18:44,2020-08-21 14:18:44,-2 days +01:54:11.895216
46,8FH5D9LXA,5f26652fd47b3c00017f4630_106,2020-08-02 14:19:35,2020-08-02 14:19:35,2020-08-14 16:32:07,2020-08-17 08:01:58,NaT,2020-08-19 23:00:00,NaT,NaN,Huyện Yên Mỹ,Hưng Yên,B,Quận 4,Hồ Chí Minh,E,Bưu Cục Yên Mỹ-Hưng Yên,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Yên Mỹ-Hưng Yên,Đang giao hàng,1000,10x10x10,2522198,NaN,'xem hàng ko nhận tt phí ship 30k- nga ',3 Ngày,1,1,0,NaN,2032281,NaN,'SHOP TÚI XÁCH HƯNG YÊN',1268482,20QFE35968,2279.0,1121.0,NaT,Xử lý giao,NaT,NaT,9999 days 00:00:00
47,8FH5DLRUS,S1412209O8412,2020-08-02 13:08:29,2020-08-02 13:08:29,2020-08-02 17:24:49,2020-08-08 08:14:42,2020-08-17 19:24:08,2020-08-17 10:37:00,NaT,'KHÁCH TỪ CHỐI NHẬN/Khách đổi ý',Huyện Đồng Hỷ,Thái Nguyên,A,Quận 7,Hồ Chí Minh,E,Bưu Cục Đồng Hỷ-Thái Nguyên,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Đồng Hỷ-Thái Nguyên,Chờ xác nhận giao lại,10,10x5x5,2525989,'08/08/2020 05:49:42 - GHN-DFC1A0|THAY ĐỔI THÔ...,'',3 Ngày,1,6,0,NaN,212432,NaN,'Công Ty Vàng Non Phương Dung',1261861,20TGY53012,2275.0,2279.0,NaT,Xử lý giao,NaT,NaT,9999 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,8FH5UDXAF,5f2dfe0f26068e0001d9ee1a_9,2020-08-08 08:43:06,2020-08-08 08:43:06,2020-08-08 14:43:24,2020-08-10 08:43:34,2020-08-17 09:57:43,2020-08-11 17:41:00,NaT,'KHÁCH TỪ CHỐI NHẬN/Khách đổi ý',Quận Cầu Giấy,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 299 Trung Kính-Q.Cầu Giấy-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 299 Trung Kính-Q.Cầu Giấy-HN,Chờ trả hàng,200,10x10x10,216531,'10/08/2020 12:05:35 - GHN-DFC1A7|THAY ĐỔI THÔ...,'MUA VÁY V79 - ĐEN:1|V79 - XANH (1)',2 Ngày,1,2,0,NaN,1872365,NaN,'Jane Shop',238188,NaN,NaN,NaN,NaT,Xử

In [26]:
# export data backlog to google sheets
# data export
data_sheets = data[[
    'MaDH',
    'MaKH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'TGNhanKien',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)

# update data to google sheets
wks.clear()
df2g.upload(
    data_sheets[data_sheets['Aging'] >= pd.Timedelta(days=0)],
    SPREADSHEET_ID, WKS_NAME,
    credentials=credentials,
    row_names=False,
)


def iter_df(df):
    """Generator for df."""
    for val in df.columns:
        yield val
    for row in df.to_numpy():
        for val in row:
            if pd.isna(val):
                yield ''
            else:
                yield val
                

# df_to_sheets(df, wks)


In [27]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)